<a href="https://colab.research.google.com/github/Marin-kh/Persian_RAG/blob/main/Persian_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install hazm

In [3]:
!pip install python-docx
!pip install rake_nltk
!pip install docx
!pip install stanza
!pip install gradio
!pip install langchain_groq
!pip install langgraph langchain_openai langchain_core

  Using cached stanza-1.10.1-py3-none-any.whl.metadata (13 kB)
  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)

In [47]:
import numpy as np
import pandas as pd
from hazm import stopwords_list, Normalizer, WordTokenizer, SentenceTokenizer, Stemmer, Lemmatizer, sent_tokenize, word_tokenize
import docx
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from sklearn.metrics.pairwise import cosine_similarity
import openai
import nltk
from rake_nltk import Rake
from google.colab import drive
import stanza
from collections import defaultdict
from openai import OpenAI
from typing import Optional, TypedDict, Annotated
from langchain_core.tools import tool, StructuredTool
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph
from langchain_core.messages import SystemMessage, AnyMessage, HumanMessage, ToolMessage
import operator
import gradio as gr

In [5]:
stanza.download('fa')
nlp = stanza.Pipeline('fa')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: fa (Persian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/fa/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package        |
------------------------------
| tokenize  | perdt          |
| mwt       | perdt          |
| pos       | perdt_charlm   |
| lemma     | perdt_nocharlm |
| depparse  | perdt_charlm   |
| ner       | arman          |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [38]:
class PersianRAKE(Rake):
    def _tokenize_text_to_sentences(self, text: str):
        return sent_tokenize(text)

    def _tokenize_sentence_to_words(self, sentence: str):
        return word_tokenize(sentence)

In [39]:
def read_from_docx(doc):
    fullText=''
    for pra in doc.paragraphs:
        fullText+=pra.text+' '

    return fullText

def split_into_overlapping_chunks(sentences, max_chunk_size=1000, overlap_size=200):
    chunks = []
    current_chunk = ""
    current_chunk_size = 0

    for sentence in sentences:
        sentence_length = len(sentence)

        if current_chunk_size + sentence_length > max_chunk_size and current_chunk:
            chunks.append(current_chunk.strip())

            overlap_buffer = current_chunk[-overlap_size:].strip() if current_chunk else ""
            current_chunk = overlap_buffer + " "
            current_chunk_size = len(overlap_buffer) + 1

        current_chunk += sentence + " "
        current_chunk_size += sentence_length + 1

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def preprocess_text_1(text):
    # text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'( +)', ' ', str(text))
    return text


def preprocess_text_2(text):
    text = re.sub('(\(.*?\))|(\[.*?\])', '', str(text))
    text = re.sub(r'( +)', ' ', str(text))

    word_tokenizer = WordTokenizer()
    words = word_tokenizer.tokenize(text)

    stopwords = stopwords_list()
    filtered_words = [word for word in words if word not in stopwords]

    lemmatizer = Lemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized_words)


def check_spelling(main_text):
    endpoint = "https://api.languagetool.org/v2/check"

    data = {
        "text": main_text,
        "language": "en-US",
    }

    response = requests.post(endpoint, data=data)
    json_response = response.json()

    updated_text = main_text

    for match in json_response.get("matches", []):
        replacement = match["replacements"][0]["value"] if match["replacements"] else ""

        offset = match["offset"]
        length = match["length"]

        updated_text = updated_text.replace(main_text[offset:offset+length], replacement)

    print("Original Query: ", main_text)
    print("Spell-checked Query: ", updated_text)
    return updated_text

def phrase_search(sentence):
    doc = nlp(sentence)

    phrases = []
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos in ['NOUN', 'ADJ']:
                phrase = word.text
                for other_word in sent.words:
                    if other_word.head == word.id and other_word.upos in ['NOUN', 'ADJ']:
                        phrase += " " + other_word.text
                if " " in phrase:
                    phrases.append(phrase)
    return phrases

def english_to_persian_number(number_str):
    english_to_persian = {
        "0": "۰",
        "1": "۱",
        "2": "۲",
        "3": "۳",
        "4": "۴",
        "5": "۵",
        "6": "۶",
        "7": "۷",
        "8": "۸",
        "9": "۹",
    }
    persian_number = "".join([english_to_persian[digit] for digit in number_str])
    return persian_number

def persian_words_to_number(sentence):
    word_to_number = {
        "صفر": 0,
        "یک": 1,
        "دو": 2,
        "سه": 3,
        "چهار": 4,
        "پنج": 5,
        "شش": 6,
        "هفت": 7,
        "هشت": 8,
        "نه": 9,
        "ده": 10,
        "یازده": 11,
        "دوازده": 12,
        "سیزده": 13,
        "چهارده": 14,
        "پانزده": 15,
        "شانزده": 16,
        "هفده": 17,
        "هجده": 18,
        "نوزده": 19,
        "بیست": 20,
        "سی": 30,
        "چهل": 40,
        "پنجاه": 50,
        "شصت": 60,
        "هفتاد": 70,
        "هشتاد": 80,
        "نود": 90,
        "صد": 100,
        "یکصد": 100,
        "دویست": 200,
        "سیصد": 300,
        "چهارصد": 400,
        "پانصد": 500,
        "ششصد": 600,
        "هفتصد": 700,
        "هشتصد": 800,
        "نهصد": 900,
        "هزار": 1000,
    }
    words = sentence.split(' ')

    result = []
    temp_number_words = []
    current_number = 0

    for word in words:
        if word[-1:]=='م' and (word[:-1] in word_to_number):
            word = word[:-1]
        if word in word_to_number:
            temp_number_words.append(word)
            current_number += word_to_number[word]
        else:
            if temp_number_words:
                english_number_str = str(current_number)
                persian_number_str = english_to_persian_number(english_number_str)
                result.append(persian_number_str)
                temp_number_words = []
                current_number = 0
            result.append(word)

    if temp_number_words:
        english_number_str = str(current_number)
        persian_number_str = english_to_persian_number(english_number_str)
        result.append(persian_number_str)

    return ' '.join(result)

def preprocess_phrases(text, phrases):
    for phrase in phrases:
        text = text.replace(phrase, phrase.replace(" ", "_"))
    return text

def extract_persian_numbers(text):
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    return re.findall(f"[{persian_digits}]+", text)

def calculate_cosine_similarity(docs, phrase):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = vectorizer.fit_transform(docs)
    phrase_vector = vectorizer.transform(phrase)
    return cosine_similarity(phrase_vector, tfidf_matrix)

def calculate_tf(document_numbers):
    tf = []
    for doc in document_numbers:
        tf_dict = defaultdict(int)
        for num in doc:
            tf_dict[num] += 1
        tf.append(tf_dict)
    return tf

def calculate_idf(document_numbers, numbers):
    idf = {}
    total_docs = len(document_numbers)
    for num in numbers:
        doc_count = sum(1 for doc in document_numbers if num in doc)
        idf[num] = np.log((total_docs + 1) / (doc_count + 1)) + 1
    return idf

def calculate_tf_idf(document_numbers, numbers):
    tf = calculate_tf(document_numbers)
    idf = calculate_idf(document_numbers, numbers)
    tf_idf = []
    for doc_tf in tf:
        doc_tf_idf = {}
        for num, freq in doc_tf.items():
            if num in idf:
                doc_tf_idf[num] = freq * idf[num]
        tf_idf.append(doc_tf_idf)
    return tf_idf

def get_city_link_by_name(persian_name: str):
    try:
        result = df_cities[df_cities['name'] == persian_name]['name_en']

        if not result.empty:
            return result.iloc[0]
        else:
            return None
    except Exception as e:
        return None

In [40]:
# Loading The Main Document
drive.mount('/content/drive')
document = read_from_docx(docx.Document("/content/drive/My Drive/delta.docx"))      #marin
# document = read_from_docx(docx.Document("/content/drive/My Drive/document.docx"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
df_api = pd.read_csv('/content/drive/My Drive/api_key.csv')
df_cities = pd.read_csv('/content/drive/My Drive/cities_per_en.csv')

In [42]:
# Chunking The Document
normalizer = Normalizer()
normalized_text = normalizer.normalize(document)

sentence_tokenizer = SentenceTokenizer()
sentences = sentence_tokenizer.tokenize(normalized_text)

max_chunk_size = 1000
overlap_size = 200
chunks = split_into_overlapping_chunks(sentences, max_chunk_size, overlap_size)
print("<Chunk 1>")
print(f"Original Chunk:\n{chunks[0]}")

# Preprocessing The Chunks
preprocessed1_chunks = [preprocess_text_1(chunk) for chunk in chunks]

preprocessed2_chunks = [preprocess_text_2(chunk) for chunk in preprocessed1_chunks]
print(f"Preprocessed Chunk:\n{preprocessed2_chunks[0]}")

<Chunk 1>
Original Chunk:
گسترش قلمرو امپراتوری ژاپن در شرق آسیا از سال ۱۹۳۱ با حمله به منچوری در شمال شرقی چین آغاز شد و در سال ۱۹۳۷ با حمله گسترده‌تر به چین ادامه یافت. در ۲۷ سپتامبر ۱۹۴۰، امپراتوری ژاپن پیمان سه‌جانبه را با آلمان و ایتالیا امضا کرد و وارد اتحاد نظامی موسوم به «نیروهای محور» شد. ایالات‌متحده آمریکا که می‌خواست توسعه‌طلبی امپراتوری ژاپن را مهار کند و آن را مجبور کند تا نیروهایش را از منچوری و چین عقب براند، علیه این کشور تحریم‌های اقتصادی وضع کرد. املاک دلتا یکی از شرکت‌های برجسته در صنعت املاک و مستغلات است که در زمینه خرید، فروش، اجاره و مدیریت املاک فعالیت می‌کند. این شرکت با بهره‌گیری از تجربه و تخصص خود در این حوزه، به مشتریان خود خدمات متنوعی ارائه می‌دهد. در این مقاله به بررسی تاریخچه، خدمات، مزایا و چالش‌های املاک دلتا خواهیم پرداخت: تاریخچه دلتا املاک دلتا در سال ۱۳۵۶ تأسیس شد و از آن زمان تا به امروز به یکی از نام‌های معتبر در صنعت املاک و مستغلات تبدیل‌شده است. این شرکت با هدف ارائه خدمات با کیفیت و ساختارهای نوآورانه به مشتریان، فعالیت خود را آغاز کرد.
Pre

In [46]:
class AgentState(TypedDict):
    input_file: Optional[str]
    messages: Annotated[list[AnyMessage], operator.add]

PROPERTY_TYPE_MAPPING = {
    "زمین": {"url": "store", "display": "زمین"},
    "آپارتمان موقعیت اداری": {"url": "OfficeLocationApartment", "display": "آپارتمان موقعیت اداری"},
    "باغ": {"url": "garden", "display": "باغ"},
    "باغچه": {"url": "garden", "display": "باغ"},
    "تجاری": {"url": "commercial", "display": "تجاری"},
    "مغازه": {"url": "commercial", "display": "مغازه"},
    "آپارتمان": {"url": "apartment", "display": "آپارتمان"}
}

def search_properties_func(min_meter: Optional[int] = None, max_meter: Optional[int] = None,
                         min_price: Optional[int] = None, max_price: Optional[int] = None,
                         city: Optional[str] = "tehran", elevator: Optional[bool] = None,
                         property_type: Optional[str] = "apartment") -> str:

    """
    جستجو بین املاک سایت با فیلتر شهر، متراژ، قیمت، آسانسور و نوع ملک.

    Args:
        min_meter (Optional[int]): حداقل متراژ مورد نظر
        max_meter (Optional[int]): حداکثر متراژ مورد نظر
        min_price (Optional[int]): حداقل قیمت مورد نظر
        max_price (Optional[int]): حداکثر قیمت مورد نظر
        city (Optional[str]): شهر مورد نظر به انگلیسی (پیش‌فرض 'tehran') یا فارسی برای تبدیل
        elevator (Optional[bool]): آیا آسانسور لازم است یا خیر
        property_type (Optional[str]): نوع ملک (مثل 'زمین', 'آپارتمان', پیش‌فرض 'apartment')

    Returns:
        str: توضیح کاربرپسند به همراه URL صفحه جستجو
    """

    if city:
        city_en = get_city_link_by_name(city)
        city = city_en if city_en else "tehran"
    else:
        city = "tehran"

    property_info = PROPERTY_TYPE_MAPPING.get(property_type, PROPERTY_TYPE_MAPPING["آپارتمان"])
    property_url = property_info["url"]
    property_display = property_info["display"]

    base_url = f"https://deltadev.ir/{city}/buy/{property_url}"


    query_params = []

    if min_meter is not None and max_meter is not None:
        query_params.append(f"meter={min_meter}-{max_meter}")
    elif min_meter is not None:
        query_params.append(f"meter={min_meter}-")
    elif max_meter is not None:
        query_params.append(f"meter=-{max_meter}")

    if min_price is not None and max_price is not None:
        query_params.append(f"price={min_price}-{max_price}")
    elif min_price is not None:
        query_params.append(f"price={min_price}-")
    elif max_price is not None:
        query_params.append(f"price=-{max_price}")

    if elevator == True:
        query_params.append("features=elavator")

    query_string = "?" + "&".join(query_params) if query_params else ""
    url = base_url + query_string

    filters = []
    if min_meter is not None and max_meter is not None:
        filters.append(f"{min_meter} تا {max_meter} متر")
    elif min_meter is not None:
        filters.append(f"حداقل {min_meter} متر")
    elif max_meter is not None:
        filters.append(f"حداکثر {max_meter} متر")

    if min_price is not None and max_price is not None:
        filters.append(f"بین {min_price:,} تا {max_price:,} تومان")
    elif min_price is not None:
        filters.append(f"حداقل {min_price:,} تومان")
    elif max_price is not None:
        filters.append(f"حداکثر {max_price:,} تومان")

    if elevator is True:
        filters.append("با آسانسور")

    if filters:
        filters_str = "، ".join(filters)
        explanation = f"با ویژگی‌هایی که گفتید ({filters_str})، می‌توانید بر روی این لینک کلیک کنید و ملک‌های مشابه را پیدا کنید: {url}"
    else:
        explanation = f"می‌توانید بر روی این لینک کلیک کنید و ملک‌های موجود را مشاهده کنید: {url}"

    return explanation

search_properties = StructuredTool.from_function(
    func=search_properties_func,
    name="search_properties",
    description="جستجو بین املاک سایت با فیلتر شهر، متراژ، قیمت، آسانسور و نوع ملک (مثل 'زمین', 'آپارتمان موقعیت اداری', 'باغ', 'تجاری'). اگر شهر مشخص نشود، تهران پیش‌فرض است. اگر نوع ملک مشخص نشود، آپارتمان پیش‌فرض است. خروجی شامل توضیح کاربرپسند و URL است.",
    args_schema=None
)

def extract_text_func(query: str) -> str:
    """
    استخراج و پردازش متن از فایل داکیومنت با استفاده از جستجوی هیبریدی و تولید پاسخ با LLM.
    Args:
        query (str): پرس‌وجوی کاربر برای استخراج اطلاعات
    Returns:
        str: پاسخ تولیدشده توسط LLM
    """

    query = check_spelling(query)
    processed_query = preprocess_text_2(preprocess_text_1(query))
    query_n = persian_words_to_number(processed_query)
    phrases = phrase_search(query_n)
    numbers = re.findall(r'\d+', query_n)
    numberic_chunks = [persian_words_to_number(chunk) for chunk in preprocessed2_chunks]

    # Similarity Of Query
    query_bonus = calculate_cosine_similarity(numberic_chunks, [query_n]).flatten()

    # Similarity Of Numbers
    document_numbers = [extract_persian_numbers(doc) for doc in numberic_chunks]
    tf_idf_numbers = calculate_tf_idf(document_numbers, numbers)

    number_bonus = np.zeros(len(numberic_chunks))
    for i, doc_tf_idf in enumerate(tf_idf_numbers):
        for num in numbers:
            if num in doc_tf_idf:
                number_bonus[i] += doc_tf_idf[num]
    if np.max(number_bonus) > 0:
        number_bonus = number_bonus / np.max(number_bonus)

    # Similarity Of Phrases
    preprocessed_docs = [preprocess_phrases(doc, phrases) for doc in numberic_chunks]
    preprocessed_phrases = preprocess_phrases(query_n, phrases)

    phrases_similarity = calculate_cosine_similarity(preprocessed_docs, [preprocessed_phrases])
    phrases_bonus = np.max(phrases_similarity, axis=0)
    if np.max(phrases_bonus) > 0:
        phrases_bonus = phrases_bonus / np.max(phrases_bonus)

    query_coef = 0.5
    numbers_coef = 0.3
    phrases_coef = 0.2

    hybrid_scores = (query_bonus * query_coef) + (number_bonus * numbers_coef) + (phrases_bonus * phrases_coef)

    top_k = 3
    indices = np.argsort(-hybrid_scores)[:top_k]

    top_3 = [chunks[idx] for idx in indices]
    while len(top_3) < 3:
        top_3.append("")

    llm = ChatGroq(
        model="llama3-70b-8192",
        api_key=df_api.loc[2, 'api_key']
    )
    prompt = f"{top_3[0]}\n{top_3[1]}\n{top_3[2]}\nطبق اطلاعات بالا، به طور خلاصه به این سوال جواب بده و از کلمات «پاراگراف»، «متن»، یا هر اشاره‌ای به منبع اطلاعات استفاده نکن. اگر اطلاعات کافی برای پاسخ وجود نداشت، دقیقاً بگو: «نمی‌توانم جواب شما را بدهم» و هیچ توضیح اضافی نده: {query}\n"

    response = llm.invoke(prompt)
    response_text = response.content.strip()

    forbidden_words = ["پاراگراف", "متن", "سند", "داکیومنت", "منبع", "بالا"]
    if any(word in response_text for word in forbidden_words) or "نمی‌توانم" in response_text and response_text != "نمی‌توانم جواب شما را بدهم":
        return "نمی‌توانم جواب شما را بدهم"
    return response_text

extract_text = StructuredTool.from_function(
    func=extract_text_func,
    name="extract_text",
    description="استخراج و پردازش متن از داکیومنت با استفاده از جستجوی هیبریدی و تولید پاسخ با LLM.",
    args_schema=None
)

def connect_to_support_func(query: str) -> str:
    """
    دادن شماره پشتیبانی سایت در صورتی که سوال مربوط به ملک، سایت دلتا یا کار کردن با سایت باشد و اطلاعات در داکیومنت موجود نباشد.
    Args:
        query (str): پرس‌وجوی کاربر
    Returns:
        str: پیام حاوی شماره پشتیبانی
    """
    return f"متأسفانه، اطلاعاتی در این باره در دسترس نیست. برای اطلاعات بیشتر، لطفاً با شماره پشتیبانی سایت دلتا (021-8686) تماس بگیرید."

connect_to_support = StructuredTool.from_function(
    func=connect_to_support_func,
    name="connect_to_support",
    description="دادن شماره پشتیبانی سایت دلتا (021-8686) زمانی که سوال کاربر درباره ملک، سایت دلتا یا نحوه کار با سایت است و اطلاعات در داکیومنت موجود نیست.",
    args_schema=None
)

def register_property_ad_func(query: str) -> str:
    """
    راهنمایی کاربر برای ثبت آگهی فروش یا اجاره ملک در سایت دلتا.
    Args:
        query (str): پرس‌وجوی کاربر
    Returns:
        str: توضیح نحوه ثبت آگهی
    """
    return "پس از ورود به حساب کاربری، با انتخاب گزینه ثبت آگهی فروشی یا ثبت آگهی اجاره و تکمیل فرم مربوطه میتوانید آگهی خود را در سایت دلتا ثبت کنید."

register_property_ad = StructuredTool.from_function(
    func=register_property_ad_func,
    name="register_property_ad",
    description="راهنمایی کاربر برای ثبت آگهی فروش یا اجاره ملک در سایت دلتا، وقتی سوال درباره فروش یا اجاره ملک است (مثل 'می‌خوام ملکم رو بفروشم' یا 'می‌خوام اجاره بدم').",
    args_schema=None
)

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=df_api.loc[2, 'api_key']
)

llm_with_tools = llm.bind_tools([search_properties, extract_text, connect_to_support, register_property_ad], parallel_tool_calls=False)

def execute_tool(tool_call):
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]
    try:
        if tool_name == "search_properties":
            return search_properties.run(tool_input=tool_args)
        elif tool_name == "extract_text":
            result = extract_text.run(tool_input=tool_args)
            if result != "نمی‌توانم جواب شما را بدهم":
                return result
            query = tool_args.get("query", "")
            if any(keyword in query for keyword in ["ملک", "خانه", "آپارتمان", "دلتا", "سایت", "زمین", "باغ", "تجاری", "مغازه"]):
                return connect_to_support.run(tool_input={"query": query})
            return result
        elif tool_name == "connect_to_support":
            return connect_to_support.run(tool_input=tool_args)
        elif tool_name == "register_property_ad":
            return register_property_ad.run(tool_input=tool_args)
        return None
    except Exception as e:
        return f"Error executing tool {tool_name}: {str(e)}"

def assistant(state: AgentState):
    image = state["input_file"]
    sys_msg = SystemMessage(content=f"""
    تو یک مشاور املاک هستی که می‌تونی با چهار ابزار زیر کمک کنی:
    - **search_properties**: برای جستجوی ملک برای خرید در سایت دلتا با فیلتر شهر، متراژ، قیمت، آسانسور و نوع ملک (مثل 'زمین', 'آپارتمان موقعیت اداری', 'باغ', 'تجاری'). فقط وقتی کاربر می‌خواد ملکی بخره (مثل 'بخرم', 'خرید') و مشخصاتی مثل شهر، متراژ، یا نوع ملک داده، از این ابزار استفاده کن. اگر شهر مشخص نشد، تهران پیش‌فرض است. اگر نوع ملک مشخص نشد، آپارتمان پیش‌فرض است. خروجی شامل توضیح کاربرپسند و URL است.
    - **extract_text**: برای استخراج اطلاعات از داکیومنت‌های بارگذاری‌شده (مثل «درباره دلتا» یا اطلاعات عمومی مثل محله‌های خوش آب و هوا).
    - **connect_to_support**: برای دادن شماره پشتیبانی سایت دلتا (021-8686) وقتی سوال درباره نحوه کار با سایت دلتا یا مواردی باشه که اطلاعاتش در داکیومنت موجود نیست و به خرید یا فروش ملک مربوط نیست.
    - **register_property_ad**: برای راهنمایی کاربر درباره ثبت آگهی فروش یا اجاره ملک در سایت دلتا، وقتی سوال درباره فروش یا اجاره ملک است (مثل 'بفروشم', 'فروختن', 'اجاره بدم').
    سوال کاربر را بخوان و ابزار مناسب را انتخاب کن:
    - اگر سوال درباره خرید ملک با مشخصات مثل شهر، متراژ، قیمت، یا نوع ملک (مثل 'یه زمین می‌خوام بخرم') است، از search_properties استفاده کن.
    - اگر سوال درباره فروش یا اجاره ملک (مثل 'می‌خوام ملکم رو بفروشم' یا 'می‌خوام اجاره بدم') است، از register_property_ad استفاده کن.
    - اگر سوال درباره اطلاعات عمومی (مثل 'خوش آب و هوا ترین محله‌ها') یا سایت دلتا است، از extract_text استفاده کن.
    - اگر extract_text پاسخی نداشت و سوال به سایت یا موارد غیرمرتبط با خرید/فروش/اجاره مربوط بود، از connect_to_support استفاده کن.
    مستند بارگذاری شده فعلی: {image}
    """)
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)

    if hasattr(response, "tool_calls") and response.tool_calls:
        tool_messages = []
        for tool_call in response.tool_calls:
            tool_result = execute_tool(tool_call)
            tool_messages.append(ToolMessage(
                content=str(tool_result),
                tool_call_id=tool_call["id"],
                name=tool_call["name"]
            ))
        return {
            "messages": [response] + tool_messages,
            "input_file": state["input_file"]
        }

    return {
        "messages": [response],
        "input_file": state["input_file"]
    }

builder = StateGraph(AgentState)
builder.add_node("assistant", assistant)
builder.set_entry_point("assistant")
graph = builder.compile()

messages = [
    # HumanMessage(content="یه زمین در شهر مشهد می‌خوام بخرم کمکم کن و ملک های مشابه رو بهم نمایش بده")
    # HumanMessage(content="خوش آب و هوا ترین محله های تهران برای خرید ملک چیه")
    HumanMessage(content="یه زمین دارم تو مشهد میخوام بفروشمش چکار کنم راهنمایی کن")

]

result = graph.invoke({"messages": messages, "input_file": None})

for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

یه زمین دارم تو مشهد میخوام بفروشمش چکار کنم راهنمایی کن
================================== Ai Message ==================================
Tool Calls:
  register_property_ad (call_dqt9)
 Call ID: call_dqt9
  Args:
    query: فروش زمین در مشهد
================================= Tool Message =================================
Name: register_property_ad

پس از ورود به حساب کاربری، با انتخاب گزینه ثبت آگهی فروشی یا ثبت آگهی اجاره و تکمیل فرم مربوطه میتوانید آگهی خود را در سایت دلتا ثبت کنید.


In [51]:
def get_answer(user_input: str) -> str:
    try:
        result = graph.invoke({"messages": [HumanMessage(content=user_input)], "input_file": None})
        # Extract the final message (usually ToolMessage or AIMessage)
        final_message = result["messages"][-1]
        return final_message.content
    except Exception as e:
        return f"خطا در پردازش درخواست: {str(e)}"


input = gr.Textbox(label="سوال خود را درباره خرید، فروش، یا اجاره ملک بپرسید")
outputs = gr.Textbox(label="")
interface = gr.Interface(
    fn=get_answer,
    inputs=input,
    outputs=outputs,
    title="دستیار هوشمند دلتا",
    clear_btn="پاک کردن",
    allow_flagging="never",
    theme="dark"
)
interface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1116: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-6818b24d-3574fa607cf9f45805d5956d;cfe44751-7fcb-4dd8-8108-ece376419ee3)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://850b43b3cc11285e7c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
